In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"navjeet69","key":"491193145107e1fa28f250bf3940adbd"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08            414  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           2364  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-30 08:31:20           1395  
balraj98/stanford-background-dataset                              Stanford Background Dataset                           17MB  2020-09-26 12:57:59            153  
nehaprabhavalkar/india

In [4]:
!kaggle datasets download -d moltean/fruits

 98% 742M/760M [00:03<00:00, 241MB/s]
100% 760M/760M [00:03<00:00, 220MB/s]


In [5]:
! mkdir data

In [ ]:
! unzip fruits.zip -d data

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [10]:
train_generator = train_datagen.flow_from_directory(
    'data/fruits-360/Training/',
    target_size=(100, 100),
    batch_size=128,
    class_mode='categorical'
)

Found 67692 images belonging to 131 classes.


In [11]:
test_generator = test_datagen.flow_from_directory(
    'data/fruits-360/Test/',
    target_size=(100, 100),
    batch_size=128,
    class_mode='categorical'
)

Found 22688 images belonging to 131 classes.


In [12]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [15]:
from keras.layers import Input, Lambda, Dense, Flatten

In [13]:
vgg = VGG16(input_shape=[100, 100, 3], include_top=False, weights='imagenet')

58892288/58889256 [==============================] - 1s 0us/step


In [14]:
for layer in vgg.layers:
  layer.trainable = False

In [18]:
x = Flatten()(vgg.output)
prediction = Dense(131, activation='softmax') (x)

In [20]:
model = keras.models.Model(inputs=vgg.input, outputs=prediction)

In [21]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)      

In [22]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop', 
    metrics=['accuracy']
)

In [24]:
model.fit_generator(
    train_generator, 
    validation_data=test_generator,
    epochs=10, 
    steps_per_epoch=67692//128,
    validation_steps=22688//128
)

Epoch 1/10
528/528 [==============================] - 76s 144ms/step - loss: 0.5206 - accuracy: 0.9201 - val_loss: 0.3012 - val_accuracy: 0.9342
Epoch 2/10
528/528 [==============================] - 75s 141ms/step - loss: 0.0118 - accuracy: 0.9992 - val_loss: 0.1951 - val_accuracy: 0.9569
Epoch 3/10
528/528 [==============================] - 74s 141ms/step - loss: 0.0024 - accuracy: 0.9998 - val_loss: 0.1877 - val_accuracy: 0.9584
Epoch 4/10
528/528 [==============================] - 75s 142ms/step - loss: 8.6737e-04 - accuracy: 1.0000 - val_loss: 0.1575 - val_accuracy: 0.9663
Epoch 5/10
528/528 [==============================] - 75s 141ms/step - loss: 4.2302e-04 - accuracy: 1.0000 - val_loss: 0.1408 - val_accuracy: 0.9725
Epoch 6/10
528/528 [==============================] - 75s 141ms/step - loss: 2.6906e-04 - accuracy: 1.0000 - val_loss: 0.1666 - val_accuracy: 0.9676
Epoch 7/10
528/528 [==============================] - 75s 141ms/step - loss: 2.0615e-04 - accuracy: 1.0000 - val_loss: